In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
tf.keras.backend.set_learning_phase(0)

In [3]:
model_v1 = tf.keras.models.load_model('./eg_ep10.h5')

/Users/fatu/venv/tensorflowpy36/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [4]:
print(model_v1.get_config())

{'name': 'sequential', 'layers': [{'class_name': 'Embedding', 'config': {'name': 'embedding_1', 'trainable': True, 'batch_input_shape': (None, 100), 'dtype': 'float32', 'input_dim': 431, 'output_dim': 32, 'embeddings_initializer': {'class_name': 'RandomUniform', 'config': {'minval': 0, 'maxval': None, 'seed': None, 'dtype': 'float32'}}, 'embeddings_regularizer': None, 'activity_regularizer': None, 'embeddings_constraint': None, 'mask_zero': False, 'input_length': 100}}, {'class_name': 'Conv1D', 'config': {'name': 'conv1d_1', 'trainable': True, 'batch_input_shape': (None, 100, 32), 'dtype': 'float32', 'filters': 64, 'kernel_size': (3,), 'strides': (1,), 'padding': 'valid', 'data_format': 'channels_last', 'dilation_rate': (1,), 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'scale': 1.0, 'mode': 'fan_avg', 'distribution': 'uniform', 'seed': None, 'dtype': 'float32'}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {'dtype

In [5]:
print(model_v1.get_weights())

[array([[ 1.0021752 ,  0.26204398,  0.31785712, ...,  0.23875196,
         0.3913663 ,  0.3587429 ],
       [ 0.81531745,  0.2572122 ,  0.54582524, ...,  0.49821588,
         0.6872999 ,  0.30481407],
       [ 0.59198606,  0.42191884, -0.09812167, ...,  0.29949376,
         0.29123694,  0.65364844],
       ...,
       [ 0.32096216,  0.32550454,  0.0472488 , ...,  0.6816348 ,
         0.39748153, -0.00397035],
       [ 0.40711677,  0.86350214,  0.41283154, ...,  0.89059377,
         0.20293581,  0.49655426],
       [ 0.98318285,  1.0639186 ,  1.3765662 , ...,  0.63009465,
         0.7101511 , -0.2548745 ]], dtype=float32), array([[[ 0.08757229, -0.09728882,  0.08866032, ...,  0.12907915,
         -0.2673875 ,  0.00167561],
        [-0.08651137,  0.00868798, -0.07626797, ..., -0.28211036,
          0.08819439, -0.01394905],
        [ 0.08739914,  0.07263602, -0.12780094, ...,  0.12506469,
         -0.32316664,  0.17936715],
        ...,
        [-0.04977933,  0.03450754, -0.06413529, ...

In [4]:
print (model_v1.outputs)

[<tf.Tensor 'dense_2/Sigmoid:0' shape=(?, 1) dtype=float32>]


In [5]:
tf.py

AttributeError: module 'tensorflow' has no attribute 'py'

In [14]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph


frozen_graph = freeze_session(tf.keras.backend.get_session(),
                              output_names=[out.op.name for out in model_v1.outputs])

INFO:tensorflow:Froze 90 variables.
INFO:tensorflow:Converted 90 variables to const ops.


In [15]:
tf.train.write_graph(frozen_graph, "model", "tf_model.pb", as_text=False)

'model/tf_model.pb'

In [3]:
f = tf.gfile.FastGFile("./model/tf_model.pb","rb")
graph_def = tf.GraphDef()

Instructions for updating:
Use tf.gfile.GFile.


In [4]:
graph_def.ParseFromString(f.read())
f.close()

In [5]:
sess = tf.Session()

In [6]:
sess.graph.as_default()
tf.import_graph_def(graph_def)

ValueError: Node 'embedding_1_1/embedding_lookup' expects to be colocated with unknown node 'embedding_1_1/embedding_lookup/Read/ReadVariableOp'